In [140]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [141]:
from sklearn.preprocessing import MinMaxScaler


In [142]:
data = pd.read_csv('PEP.csv')


In [143]:
data

,ticker,date,open,high,low,close
0,PEP,1972-06-01,0.000,1.60,1.580,1.60
1,PEP,1972-06-02,1.600,1.62,1.600,1.60
2,PEP,1972-06-05,1.600,1.60,1.540,1.57
3,PEP,1972-06-06,1.560,1.56,1.560,1.56
4,PEP,1972-06-07,1.560,1.56,1.550,1.55
...,...,...,...,...,...,...
12826,PEP,2023-04-13,182.620,184.61,182.105,184.38
12827,PEP,2023-04-14,183.335,184.02,182.920,183.51
12828,PEP,2023-04-17,184.190,184.75,183.481,184.45
12829,PEP,2023-04-18,184.770,185.22,183.740,184.83


In [144]:
data = data.filter(['close'])


In [145]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
11831,127.24
11832,126.83
11833,127.67
11834,127.27
11835,125.98
...,...
12826,184.38
12827,183.51
12828,184.45
12829,184.83


In [146]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [147]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [148]:

data = train_data

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]


In [149]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [150]:
start_time = time.time()

# Tạo một list chứa các số Lucas để sử dụng làm số lượng hidden layer
lucas_list = [3, 4, 7, 11, 18, 29, 47, 76]

# Chọn số lượng hidden layer sử dụng theo số Lucas
num_hidden_layer = lucas_list[0] # ví dụ chọn số 11

In [151]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(num_hidden_layer): #Thêm num_hidden_layer lớp ẩn với 64 đơn vị mỗi lớp
    model.add(Dense(units=100, activation='relu')) #Tham số chỉ định số lượng tế bào thần kinh trong lớp
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [152]:
history = model.fit(X_train, y_train, epochs=11000, batch_size=8)

Epoch 1/9000
79/79 [==============================] - 1s 2ms/step - loss: 0.0273
Epoch 2/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 3/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 4/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 5/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0048
Epoch 6/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0044
Epoch 7/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 8/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 9/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 10/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 11/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0030
Epoch 12/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 13/9000
79/79 [====

79/79 [==============================] - 0s 2ms/step - loss: 7.8680e-04
Epoch 197/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.0914e-04
Epoch 198/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.8736e-04
Epoch 199/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.2061e-04
Epoch 200/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.1986e-04
Epoch 201/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.0430e-04
Epoch 202/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.3611e-04
Epoch 203/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.2482e-04
Epoch 204/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.4324e-04
Epoch 205/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.8993e-04
Epoch 206/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.3600e-04
Epoch 207/9000
79/79 [==============================] - 0s

79/79 [==============================] - 0s 2ms/step - loss: 8.3457e-04
Epoch 385/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.0938e-04
Epoch 386/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.2105e-04
Epoch 387/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.9280e-04
Epoch 388/9000
79/79 [==============================] - 0s 2ms/step - loss: 9.2841e-04
Epoch 389/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.7142e-04
Epoch 390/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.6165e-04
Epoch 391/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.9356e-04
Epoch 392/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.0411e-04
Epoch 393/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.6077e-04
Epoch 394/9000
79/79 [==============================] - 0s 2ms/step - loss: 8.4737e-04
Epoch 395/9000
79/79 [==============================] - 0s

79/79 [==============================] - 0s 2ms/step - loss: 7.7253e-04
Epoch 573/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.4672e-04
Epoch 574/9000
79/79 [==============================] - 0s 3ms/step - loss: 7.2231e-04
Epoch 575/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.7347e-04
Epoch 576/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.8340e-04
Epoch 577/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.3674e-04
Epoch 578/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.2643e-04
Epoch 579/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.2104e-04
Epoch 580/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.5567e-04
Epoch 581/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.6022e-04
Epoch 582/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.3079e-04
Epoch 583/9000
79/79 [==============================] - 0s

79/79 [==============================] - 0s 2ms/step - loss: 6.9729e-04
Epoch 761/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.9517e-04
Epoch 762/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.7672e-04
Epoch 763/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.7145e-04
Epoch 764/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.7876e-04
Epoch 765/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.5750e-04
Epoch 766/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.4400e-04
Epoch 767/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.4604e-04
Epoch 768/9000
79/79 [==============================] - 0s 2ms/step - loss: 7.1505e-04
Epoch 769/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.8140e-04
Epoch 770/9000
79/79 [==============================] - 0s 2ms/step - loss: 6.4413e-04
Epoch 771/9000
79/79 [==============================] - 0s

...

79/79 [==============================] - 0s 2ms/step - loss: 4.4083e-04
Epoch 8203/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.6193e-04
Epoch 8204/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.8725e-04
Epoch 8205/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5388e-04
Epoch 8206/9000
79/79 [==============================] - 0s 2ms/step - loss: 3.8897e-04
Epoch 8207/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.7874e-04
Epoch 8208/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.1817e-04
Epoch 8209/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.1708e-04
Epoch 8210/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.8909e-04
Epoch 8211/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5707e-04
Epoch 8212/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.3192e-04
Epoch 8213/9000
79/79 [=========================

79/79 [==============================] - 0s 2ms/step - loss: 5.2046e-04
Epoch 8389/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9788e-04
Epoch 8390/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.6951e-04
Epoch 8391/9000
79/79 [==============================] - 0s 2ms/step - loss: 3.9695e-04
Epoch 8392/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.6636e-04
Epoch 8393/9000
79/79 [==============================] - 0s 2ms/step - loss: 3.9396e-04
Epoch 8394/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.7149e-04
Epoch 8395/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.3742e-04
Epoch 8396/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9928e-04
Epoch 8397/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.4021e-04
Epoch 8398/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.0675e-04
Epoch 8399/9000
79/79 [=========================

79/79 [==============================] - 0s 2ms/step - loss: 5.0436e-04
Epoch 8575/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.0793e-04
Epoch 8576/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5390e-04
Epoch 8577/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.1761e-04
Epoch 8578/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.0690e-04
Epoch 8579/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.4966e-04
Epoch 8580/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5925e-04
Epoch 8581/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.8190e-04
Epoch 8582/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.2922e-04
Epoch 8583/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.6505e-04
Epoch 8584/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5839e-04
Epoch 8585/9000
79/79 [=========================

79/79 [==============================] - 0s 2ms/step - loss: 5.7180e-04
Epoch 8761/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.5702e-04
Epoch 8762/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9058e-04
Epoch 8763/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.0080e-04
Epoch 8764/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9239e-04
Epoch 8765/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9636e-04
Epoch 8766/9000
79/79 [==============================] - 0s 2ms/step - loss: 3.9742e-04
Epoch 8767/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.7208e-04
Epoch 8768/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.5535e-04
Epoch 8769/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.9888e-04
Epoch 8770/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.0833e-04
Epoch 8771/9000
79/79 [=========================

79/79 [==============================] - 0s 2ms/step - loss: 4.2783e-04
Epoch 8947/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.4691e-04
Epoch 8948/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.5183e-04
Epoch 8949/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.6842e-04
Epoch 8950/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.8965e-04
Epoch 8951/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.0078e-04
Epoch 8952/9000
79/79 [==============================] - 0s 2ms/step - loss: 3.9210e-04
Epoch 8953/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.5355e-04
Epoch 8954/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.4536e-04
Epoch 8955/9000
79/79 [==============================] - 0s 2ms/step - loss: 5.4986e-04
Epoch 8956/9000
79/79 [==============================] - 0s 2ms/step - loss: 4.4931e-04
Epoch 8957/9000
79/79 [=========================

In [153]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

5/5 [==============================] - 0s 3ms/step
Training time: 1579.708600282669 seconds


In [154]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAPE: ', mape)

5/5 [==============================] - 0s 5ms/step
MSE:  0.0035276790321002683
RMSE:  0.05939426767037597
MAPE:  0.06597929141614972


In [155]:
len(data)

800